In [1]:
#Se importan las librerías necesarias para el problema
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/df_procesado.csv')

C:\Users\JhonG\AppData\Local\Temp\ipykernel_22180\2713026249.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/df_procesado.csv')


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import roc_auc_score
from numpy import mean
from sklearn.ensemble import RandomForestClassifier
import wandb


In [4]:
data = shuffle(df[['OPERA', 'MES', 'periodo_dia','TIPOVUELO', 'SIGLADES', 'DIANOM', 'atraso_15']], random_state = 111)

In [5]:
features = pd.concat([pd.get_dummies(data['OPERA'], prefix = 'OPERA'),pd.get_dummies(data['TIPOVUELO'], prefix = 'TIPOVUELO'),pd.get_dummies(data['periodo_dia'], prefix = 'periodo_dia'), pd.get_dummies(data['MES'], prefix = 'MES')], axis = 1)
label = data['atraso_15']

In [6]:
features_importantes = features[['MES_7', 'TIPOVUELO_I', 'OPERA_Copa Air', 'OPERA_Latin American Wings',
                                'MES_12', 'OPERA_Grupo LATAM', 'MES_10', 'OPERA_JetSmart SPA', 'OPERA_Air Canada',
                                'MES_9', 'OPERA_American Airlines']]
labels_ = label

In [7]:
df_features_importantes = pd.concat([features_importantes, label], axis =1)
df_features_importantes

,MES_7,TIPOVUELO_I,OPERA_Copa Air,OPERA_Latin American Wings,MES_12,OPERA_Grupo LATAM,MES_10,OPERA_JetSmart SPA,OPERA_Air Canada,MES_9,OPERA_American Airlines,atraso_15
13375,0,0,0,0,0,0,0,0,0,0,0,0
35799,1,0,0,0,0,1,0,0,0,0,0,0
48176,0,0,0,0,0,1,0,0,0,1,0,0
58675,0,0,0,0,0,1,0,0,0,0,0,0
23374,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
37831,1,1,0,0,0,1,0,0,0,0,0,0
28790,0,0,0,0,0,0,0,0,0,0,0,1
25257,0,0,0,0,0,1,0,0,0,0,0,0
4820,0,1,0,0,0,1,0,0,0,0,0,0


In [8]:
df_features_importantes.to_csv("df_features_importantes.csv", index = False)

# PIPELINE DE MONITOREO
Para probar distintos modelos y comparar sus métricas de forma cómoda, es buena práctica usar librerías que permitan hacer seguimiento de los distintos modelos entrenados, a modo de poder comparar y seleccionar correctamente los modelos de forma compacta. En este caso hacemos uso de W&B

Dentro de la pipeline se incluyen procesos como Cross Validation, Oversampling, Undersampling.

# UN-UPSAMPLING Todas las features

In [8]:
#creamos un diccionario con modelos base clasicos para clasificacion que seran sometidos a test

models = {
    'SVM' : SVC(),
    'XGB' : xgb.XGBClassifier(random_state=1, learning_rate=0.01),
    'RF'  : RandomForestClassifier(random_state = 1),
    'LR'  : LogisticRegression()
 }


In [7]:
#nuestro primer hiperparametro a testear es el under-over-sampling para la data desbalanceada.
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters': 
    {
        'over_values': {'values': [0.3, 0.4, 0.5]},
        'under_values': {'values': [0.7, 0.6, 0.5]}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project='cw-challenge-model-test')


Create sweep with ID: aiz2vglq
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-test/sweeps/aiz2vglq


In [8]:
import wandb

X = features
y = label

def train():
    wandb.init()
    for name, model in models.items():
        # define pipeline
        over = SMOTE(sampling_strategy=wandb.config.over_values)
        under = RandomUnderSampler(sampling_strategy=wandb.config.under_values)
        steps = [('over', over), ('under', under), ('model', model)]
        pipeline = Pipeline(steps=steps)
        # evaluate pipeline
        scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
        #score = mean(scores)
            
        wandb.log({
            'accuracy_'+name: mean(scores['test_accuracy']),
            'f1_'+name : mean(scores['test_f1']),
            'precision_'+name : mean(scores['test_precision']),
            'recall_'+name : mean(scores['test_recall']),
            'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
        print("Modelo: ", name)
        #print(scores)
wandb.agent(sweep_id, function = train)

wandb: Agent Starting Run: 2pwh13jl with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.7
wandb: Currently logged in as: ignacio-onate. Use `wandb login --relogin` to force relogin


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 1ssmyda8 with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.6


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


...

accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 7gmctfvl with config:
wandb: 	over_values: 0.5
wandb: 	under_values: 0.5


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# UN-UPSAMPLING Features importantes

In [12]:
sweep_configuration2 = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters': 
    {
        'over_values': {'values': [0.3, 0.4, 0.5]},
        'under_values': {'values': [0.7, 0.6, 0.5]}
     }
}
sweep_id2 = wandb.sweep(sweep=sweep_configuration2, project='cw-challenge-model-test-important-features')

Create sweep with ID: x56k4uit
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-test-important-features/sweeps/x56k4uit


In [13]:
import wandb

X = features_importantes
y = labels_

def train():
    wandb.init()
    for name, model in models.items():
        # define pipeline
        over = SMOTE(sampling_strategy=wandb.config.over_values)
        under = RandomUnderSampler(sampling_strategy=wandb.config.under_values)
        steps = [('over', over), ('under', under), ('model', model)]
        pipeline = Pipeline(steps=steps)
        # evaluate pipeline
        scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
        #score = mean(scores)
            
        wandb.log({
            'accuracy_'+name: mean(scores['test_accuracy']),
            'f1_'+name : mean(scores['test_f1']),
            'precision_'+name : mean(scores['test_precision']),
            'recall_'+name : mean(scores['test_recall']),
            'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
        print("Modelo: ", name)
        #print(scores)
#
wandb.agent(sweep_id2, function = train)

wandb: Agent Starting Run: 2v0u3eyv with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.7
wandb: Currently logged in as: ignacio-onate. Use `wandb login --relogin` to force relogin


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: 5kx86287 with config:
wandb: 	over_values: 0.3
wandb: 	under_values: 0.6


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


...

accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Agent Starting Run: js0vesuh with config:
wandb: 	over_values: 0.5
wandb: 	under_values: 0.5


Modelo:  SVM
Modelo:  XGB
Modelo:  RF
Modelo:  LR


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# UNSAMPLING


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
under_sampler = RandomUnderSampler(random_state=42)

X = features
y = label



wandb.init(project="Unsampling")
for name, model in models.items():
    # define pipeline
    under = RandomUnderSampler(random_state=42)
    steps = [('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)
            
    wandb.log({
        'accuracy_'+name: mean(scores['test_accuracy']),
        'f1_'+name : mean(scores['test_f1']),
        'precision_'+name : mean(scores['test_precision']),
        'recall_'+name : mean(scores['test_recall']),
        'roc_auc_'+name : mean(scores['test_roc_auc'])
    })
    print(name +' accuracy: ', mean(scores['test_accuracy']))
    print(name +' f1: ', mean(scores['test_f1']))
    print(name + ' precision: ', mean(scores['test_precision']))
    print(name + ' recall: ', mean(scores['test_recall']))
    print(name+ ' roc_auc:', mean(scores['test_roc_auc']))
    print("Modelo: ", name)
    #print(scores)


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


In [10]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
under_sampler = RandomUnderSampler(random_state=42)

X = features_importantes
y = labels_



wandb.init(project="Unsampling-fi")
for name, model in models.items():
    # define pipeline
    under = RandomUnderSampler(random_state=42)
    steps = [('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)
            
    wandb.log({
        'accuracy_'+name: mean(scores['test_accuracy']),
        'f1_'+name : mean(scores['test_f1']),
        'precision_'+name : mean(scores['test_precision']),
        'recall_'+name : mean(scores['test_recall']),
        'roc_auc_'+name : mean(scores['test_roc_auc'])
    })
    print(name +' accuracy: ', mean(scores['test_accuracy']))
    print(name +' f1: ', mean(scores['test_f1']))
    print(name + ' precision: ', mean(scores['test_precision']))
    print(name + ' recall: ', mean(scores['test_recall']))
    print(name+ ' roc_auc:', mean(scores['test_roc_auc']))
    print("Modelo: ", name)
    #print(scores)


accuracy_LR,▁
accuracy_RF,▁
accuracy_SVM,▁
accuracy_XGB,▁
f1_LR,▁
f1_RF,▁
f1_SVM,▁
f1_XGB,▁
precision_LR,▁
precision_RF,▁
precision_SVM,▁


SVM accuracy:  0.5522381639619454
SVM f1:  0.3597763419482513
SVM precision:  0.24487483069157281
SVM recall:  0.6805146812685703
SVM roc_auc: 0.607959027156658
Modelo:  SVM
XGB accuracy:  0.5373867756682418
XGB f1:  0.3610332485999721
XGB precision:  0.24245412810141959
XGB recall:  0.7066744868698851
XGB roc_auc: 0.6386347923725102
Modelo:  XGB
RF accuracy:  0.5567539619239716
RF f1:  0.35821883630553936
RF precision:  0.24488018854420557
RF recall:  0.6690189511058138
RF roc_auc: 0.6415320812637301
Modelo:  RF
LR accuracy:  0.5369909586260391
LR f1:  0.3582661431902333
LR precision:  0.24088115942061558
LR recall:  0.6989043857572977
LR roc_auc: 0.6349135728706592
Modelo:  LR


# AJUSTE DE HIPERPARAMETROS
Tras los experimentos podemos concluir que el proceso que mejor balancea los datos el undersampling, quizas el accuracy no es el mas alto en los modelos, pero sus otras metricas son las mas altas.

En el caso de RandomForest con todas las features, logra un mejor desempeño que los otros modelos segun la metrica Recall. Que es interesante para medir que tanto se equivoca al predecir como atraso. Por lo que tambien es interesante probar sus hiperparametros.



# Random Forest

In [11]:
#Para esto debemos crear los diccionarios de hiperparametros.
sweep_configuration3 = {
    'method': 'bayes', #Usando Bayes como metodo de busqueda de parametros
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters': 
    {
        'bootstrap': {'values':[True, False]},
        'max_depth': {'values' :[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]},
        'max_features': {'values':['auto', 'sqrt']},
        'min_samples_leaf': {'values': [1, 2, 4]},
        'min_samples_split': {'values': [2, 5, 10]},
        'n_estimators': {'values' : [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
     }
}
sweep_id3 = wandb.sweep(sweep=sweep_configuration3, project='cw-challenge-model-RF-ht')

Create sweep with ID: wc95eubb
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-RF-ht/sweeps/wc95eubb


In [12]:
X = features
y = label

def train():
    wandb.init()
    # define pipeline
    name = 'RF'
    model = RandomForestClassifier(bootstrap = wandb.config.bootstrap, max_depth=wandb.config.max_depth, max_features = wandb.config.max_features)
    under = RandomUnderSampler(sampling_strategy=0.7)
    steps = [('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)         
    wandb.log({
        'accuracy_'+name: mean(scores['test_accuracy']),
        'f1_'+name : mean(scores['test_f1']),
        'precision_'+name : mean(scores['test_precision']), 
        'recall_'+name : mean(scores['test_recall']),
        'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
    print("Modelo: ", name)
    #print(scores)
wandb.agent(sweep_id3, function = train, count = 15)

wandb: Agent Starting Run: ofn7vf0y with config:
wandb: 	bootstrap: False
wandb: 	max_depth: 10
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 200


Modelo:  RF


accuracy_RF,▁
f1_RF,▁
precision_RF,▁
recall_RF,▁
roc_auc_RF,▁
accuracy_RF,0.7838
f1_RF,0.31118
precision_RF,0.38021
recall_RF,0.26455
roc_auc_RF,0.67654


wandb: Agent Starting Run: br4qf822 with config:
wandb: 	bootstrap: True
wandb: 	max_depth: 50
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 10
wandb: 	n_estimators: 800


Modelo:  RF


...

Modelo:  RF


accuracy_RF,▁
f1_RF,▁
precision_RF,▁
recall_RF,▁
roc_auc_RF,▁
accuracy_RF,0.72913
f1_RF,0.37177
precision_RF,0.32565
recall_RF,0.43349
roc_auc_RF,0.68183


wandb: Agent Starting Run: 7dmyjxnh with config:
wandb: 	bootstrap: False
wandb: 	max_depth: 50
wandb: 	max_features: auto
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 800


Modelo:  RF


accuracy_RF,▁
f1_RF,▁
precision_RF,▁
recall_RF,▁
roc_auc_RF,▁
accuracy_RF,0.73432
f1_RF,0.36411
precision_RF,0.32692
recall_RF,0.41176
roc_auc_RF,0.68051


# XGB con todas las features

In [13]:
#Para esto debemos crear los diccionarios de hiperparametros.
sweep_configuration4 = {
    'method': 'bayes', #Usando Bayes como metodo de busqueda de parametros
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters' : {
                'learning_rate': {'values':[0.01,0.05, 0.1]},
                'n_estimators': {'values' : [50, 100, 150]},
                'subsample': {'values' : [0.5, 0.9]}
    }
}
sweep_id4 = wandb.sweep(sweep=sweep_configuration4, project='cw-challenge-model-XGB-ht')

Create sweep with ID: lxyem30k
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-XGB-ht/sweeps/lxyem30k


In [14]:
X = features
y = label

def train():
    wandb.init()
    # define pipeline
    name = 'XGB_FP'
    model = xgb.XGBClassifier(random_state=1, learning_rate=wandb.config.learning_rate, subsample = wandb.config.subsample, n_estimators=wandb.config.n_estimators,max_depth = 10)    
    under = RandomUnderSampler()
    steps = [('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)         
    wandb.log({
        'accuracy_'+name: mean(scores['test_accuracy']),
        'f1_'+name : mean(scores['test_f1']),
        'precision_'+name : mean(scores['test_precision']), 
        'recall_'+name : mean(scores['test_recall']),
        'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
    print("Modelo: ", name)
    #print(scores)
wandb.agent(sweep_id4, function = train, count = 15)

wandb: Agent Starting Run: s5vup0m1 with config:
wandb: 	learning_rate: 0.05
wandb: 	n_estimators: 50
wandb: 	subsample: 0.9


Modelo:  XGB_FP


accuracy_XGB_FP,▁
f1_XGB_FP,▁
precision_XGB_FP,▁
recall_XGB_FP,▁
roc_auc_XGB_FP,▁
accuracy_XGB_FP,0.61223
f1_XGB_FP,0.38818
precision_XGB_FP,0.27411
recall_XGB_FP,0.66506
roc_auc_XGB_FP,0.68196


wandb: Agent Starting Run: 4dtyfi34 with config:
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 150
wandb: 	subsample: 0.9


Modelo:  XGB_FP


...

Modelo:  XGB_FP


accuracy_XGB_FP,▁
f1_XGB_FP,▁
precision_XGB_FP,▁
recall_XGB_FP,▁
roc_auc_XGB_FP,▁
accuracy_XGB_FP,0.62852
f1_XGB_FP,0.38659
precision_XGB_FP,0.27839
recall_XGB_FP,0.63295
roc_auc_XGB_FP,0.68172


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: is1wghwx with config:
wandb: 	learning_rate: 0.1
wandb: 	n_estimators: 100
wandb: 	subsample: 0.9


Modelo:  XGB_FP


accuracy_XGB_FP,▁
f1_XGB_FP,▁
precision_XGB_FP,▁
recall_XGB_FP,▁
roc_auc_XGB_FP,▁
accuracy_XGB_FP,0.62987
f1_XGB_FP,0.38969
precision_XGB_FP,0.28037
recall_XGB_FP,0.63905
roc_auc_XGB_FP,0.68315


# XGB Features principales

In [15]:
#Para esto debemos crear los diccionarios de hiperparametros.
sweep_configuration5 = {
    'method': 'bayes', #Usando Bayes como metodo de busqueda de parametros
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'precision'},
    'parameters' : {
                'learning_rate': {'values':[0.01,0.05, 0.1]},
                'n_estimators': {'values' : [50, 100, 150]},
                'subsample': {'values' : [0.5, 0.9]}
    }
}
sweep_id5 = wandb.sweep(sweep=sweep_configuration4, project='cw-challenge-model-XGB-ht-fp')

Create sweep with ID: 0lcka3ii
Sweep URL: https://wandb.ai/ignacio-onate/cw-challenge-model-XGB-ht-fp/sweeps/0lcka3ii


In [16]:
X = features_importantes
y = labels_

def train():
    wandb.init()
    # define pipeline
    name = 'XGB_FI'
    model = xgb.XGBClassifier(random_state=1, learning_rate=wandb.config.learning_rate, subsample = wandb.config.subsample, n_estimators=wandb.config.n_estimators,max_depth = 10)    
    under = RandomUnderSampler()
    steps = [('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)         
    wandb.log({
        'accuracy_'+name: mean(scores['test_accuracy']),
        'f1_'+name : mean(scores['test_f1']),
        'precision_'+name : mean(scores['test_precision']), 
        'recall_'+name : mean(scores['test_recall']),
        'roc_auc_'+name : mean(scores['test_roc_auc'])
        })
    print("Modelo: ", name)
    #print(scores)
wandb.agent(sweep_id5, function = train, count = 15)

wandb: Agent Starting Run: fah8wym1 with config:
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 50
wandb: 	subsample: 0.9


Modelo:  XGB_FI


accuracy_XGB_FI,▁
f1_XGB_FI,▁
precision_XGB_FI,▁
recall_XGB_FI,▁
roc_auc_XGB_FI,▁
accuracy_XGB_FI,0.55134
f1_XGB_FI,0.35917
precision_XGB_FI,0.24461
recall_XGB_FI,0.67933
roc_auc_XGB_FI,0.64191


wandb: Agent Starting Run: a0zgelr5 with config:
wandb: 	learning_rate: 0.1
wandb: 	n_estimators: 150
wandb: 	subsample: 0.9


Modelo:  XGB_FI


...

Modelo:  XGB_FI


accuracy_XGB_FI,▁
f1_XGB_FI,▁
precision_XGB_FI,▁
recall_XGB_FI,▁
roc_auc_XGB_FI,▁
accuracy_XGB_FI,0.57945
f1_XGB_FI,0.35812
precision_XGB_FI,0.2499
recall_XGB_FI,0.6343
roc_auc_XGB_FI,0.64151


wandb: Agent Starting Run: iky1tqj8 with config:
wandb: 	learning_rate: 0.05
wandb: 	n_estimators: 100
wandb: 	subsample: 0.9


Modelo:  XGB_FI


accuracy_XGB_FI,▁
f1_XGB_FI,▁
precision_XGB_FI,▁
recall_XGB_FI,▁
roc_auc_XGB_FI,▁
accuracy_XGB_FI,0.54431
f1_XGB_FI,0.3594
precision_XGB_FI,0.24317
recall_XGB_FI,0.69161
roc_auc_XGB_FI,0.64216


# CONCLUSIONES PARA LAS METRICAS OBTENIDAS
Si bien, los resultados no fueron alentadores, para continuar el trabajo sobre este set de datos, hay que tener en cuenta el objetivo de esta predicción, por ejemplo, en caso de ser de uso logistico donde predecir un atraso positivo implica una preparacion extra por parte del aeropuerto la cual consume recursos, es escencial buscar aumentar la metrica que nos permita disminuir el mal uso de los recursos. En caso que sea para informar a un cliente si su vuelo podria atrasarse, en este caso, el predecir un positivo podria no ser costoso para el aeropuerto, por lo que maximizar otras metricas podria ser mejor.

### Para continuar con el desafio se esocgio el modelo XGB con los hiperparametros encontrados y todas las features.

In [18]:
#Entrenamos un nuevo modelo, con toda la data y los hiperparametros seleccionados en el sweep con mejores resultados.
X = features
y= label
model = xgb.XGBClassifier(random_state=1, learning_rate=0.01, subsample = 0.5, n_estimators=100, max_depth = 10)
model.fit(X,y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [19]:
import pickle
pickle.dump(model, open('XGB-model-cw.pkl', 'wb'))
